# Solving a different problem: uncertainty-constrained temperature maximization

## Temperature as the objective

What if, instead of minimizing wind deviations, we fixed a wind forecast inaccuracy limit (placing bounds on deviation variables) and maximized heat input to each line? This would give us a set of wind patterns, each one based on $x\%$ maximum forecast inaccuracy, that would maximally heat one line in the network. We could then filter that list based on which line temperatures are greatest, and sort remaining patterns according to distance from the forecast. In the end, we would have the most likely wind pattern (assuming inaccuracy of $x\%$) guaranteed to saturate a line.

We get a quadratic program like this:

\begin{align}
&& z_{ik}^* = \max~ & z^\top Q_{\theta} z \\
s.t. && Az &= b \\
&& x &\leq c\cdot P0  \\
&& x &\geq -c\cdot P0
\end{align}

where $z$ is the vector of all variables, $x$ contains only wind deviation variables, and $P0$ is the vector of wind forecast values. We define $A$ to ensure the right line $(i,k)$ is chosen, and we repeat the optimization to find $z_{ik}^*$ for every line in the network. This gives us a set of system states (including wind deviations), each of which reflects maximal heating of one line in the network subject to the given forecast inaccuracy.

One nice side-effect of this formulation is that we don't have to worry about wind farm output going negative. Deviations are bounded, so they cannot exceed forecast values as long as $c\leq 1$.

## Understanding the output

Suppose $z_{ik}^*$ results in a temperature of $T_{ik}^*$ for line $(i,k)$, and the line's maximum allowable temperature is $T_{ik}^{max}$. If $T_{ik}^* \geq T_{ik}^{max}$, we have a problem. Otherwise, there is no wind pattern within the forecast inaccuracy limits that will excessively heat line $(i,k)$. We remove $z_{ik}^*$ from our list and move on.

In the end we have a set of problematic system states. If this set is empty, we need to increase the inaccuracy limit and re-do the optimization. Otherwise, we rank members of the set according to $\lVert x_{ik}^*\rVert$, the normed distance between the forecast and the actual wind pattern. This gives us a set of "extreme events" sorted by pseudo-likelihood (since patterns closer to the forecast are more likely to occur).

## Reusing existing code

I can reuse many elements of code I have already written for the PowerTech temporal instanton problem.

* $Q_\theta$ selects only the $\hat{\theta}_{ik}$ variables from $z$. This is the same function it has in the original problem, so the code doesn't need alteration.

* $Ax=b$ ensures power balance and implements distributed slack. It also defines $\hat{\theta}_{ik}$ appropriately. I do not need to change this at all.

* All I need to do is add constraints to limit wind deviations (otherwise, the objective could increase without bound).

## Preliminary results

I used Ipopt to solve this "maxTemp" problem with RTS-96 data. After playing with various parameters and solving across all lines dozens of times, Ipopt has yet to return anything but an `:Optimal` status. The results satisfy $Az=b$, and as I increase $c$ I get greater objective values, as is intuitively expected. Mismatch values make sense and respond according to generation/demand scaling as expected. If anything, this experiment has been a good test for the modeling portion of my code (`TemporalInstanton` module).

I cannot say whether the solutions it is producing are globally optimal without studying the problem further. Though norm maximization is NP-hard in general, [it can exhibit strong duality](http://web.stanford.edu/class/cme334/docs/2011-11-17-Rubira_normmax.pdf).